In [1]:
import keras_preprocessing.sequence
import numpy as np
import pandas as pd
import re
import time
import random
import tensorflow as tf
from keras import layers
import tensorflow_datasets as tfds
from util.limpeza import limpeza
import multiprocessing as mp

## base de dados

In [2]:
with open('./tradutor/europarl-v7.pt-en.en', mode='r', encoding='utf-8') as f:
    europarl_en = f.read()

with open('./tradutor/europarl-v7.pt-en.pt', mode='r', encoding='utf-8') as f:
    europarl_pt = f.read()

corpus_en = europarl_en
corpus_pt = europarl_pt

In [3]:
corpus_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_en)
corpus_en = re.sub(r".\$\$\$", '', corpus_en)
corpus_en = re.sub(r" +", ' ', corpus_en)
corpus_en = corpus_en.split('\n')
print(len(corpus_en))

corpus_pt = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_pt)
corpus_pt = re.sub(r".\$\$\$", '', corpus_pt)
corpus_pt = re.sub(r" +", ' ', corpus_pt)
corpus_pt = corpus_pt.split('\n')
print(len(corpus_pt))

1960408
1960408


## tokenizer 

In [4]:
#### tokenizer ####
# build_tokenize_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(corpus_en, target_vocab_size=2**13)
# build_tokenize_pt = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(corpus_pt, target_vocab_size=2**13)
#
# build_tokenize_en.save_to_file("./vocab_tokenizer_en")
# build_tokenize_pt.save_to_file("./vocab_tokenizer_pt")


In [5]:
print("carregando o vocab")

tokenize_en = tfds.deprecated.text.SubwordTextEncoder.load_from_file('./vocabs/vocab_tokenizer_en')
tokenize_pt = tfds.deprecated.text.SubwordTextEncoder.load_from_file('./vocabs/vocab_tokenizer_pt')

print("vocab carregado")

carregando o vocab
vocab carregado


In [6]:
vocab_size_en = tokenize_en.vocab_size + 2
inputs_en = [[vocab_size_en - 2] + tokenize_en.encode(sentence) + [vocab_size_en - 1] for sentence in corpus_en]
print(inputs_en[random.randint(0, len(inputs_en) - 1)])

vocab_size_pt = tokenize_pt.vocab_size + 2
outputs_pt = [[vocab_size_pt - 2] + tokenize_pt.encode(sentence) + [vocab_size_pt - 1] for sentence in corpus_pt]
print(outputs_pt[random.randint(0, len(outputs_pt) - 1)])

[8191, 3014, 55, 159, 233, 75, 107, 497, 7980, 403, 7980, 1744, 260, 12, 120, 2, 87, 11, 137, 8, 9, 7, 858, 193, 3, 16, 457, 7981, 8192]
[8116, 25, 475, 459, 498, 75, 1336, 2831, 6, 443, 242, 1, 4, 2615, 7892, 3, 992, 23, 3863, 134, 21, 234, 26, 961, 8, 282, 8, 7227, 3260, 20, 599, 66, 7957, 75, 4638, 419, 2, 28, 1689, 7070, 6, 10, 207, 2, 147, 5, 62, 10, 147, 3, 243, 2383, 1804, 1, 251, 27, 14, 484, 7, 1768, 1929, 2, 1689, 10, 62, 1696, 1, 7958, 75, 484, 7, 1429, 2, 788, 15, 7568, 19, 1551, 34, 5623, 1, 7959, 75, 5373, 2, 2948, 4443, 7892, 26, 317, 15, 711, 5220, 9, 26, 599, 1, 7960, 75, 2731, 2, 28, 7301, 226, 1, 7961, 75, 4026, 5, 7, 457, 2, 2918, 2453, 1965, 23, 733, 8, 7227, 3260, 10, 62, 2021, 200, 743, 5047, 488, 4625, 7957, 7906, 8117]


## etapa de melhora do processamento

In [7]:
"""
o Objetivo é tirar frases com mais de 15 palavras
"""

max_length = 15
idx_to_remove = [count for count, sent in enumerate(inputs_en) if len(sent) > max_length]
for idx in reversed(idx_to_remove):
    del inputs_en[idx]
    del outputs_pt[idx]

idx_to_remove = [count for count, sent in enumerate(outputs_pt) if len(sent) > max_length]
for idx in reversed(idx_to_remove):
    del inputs_en[idx]
    del outputs_pt[idx]

etapa de melhora do processamento


In [8]:
#### tamanho total
print("tamanhos das entradas e saidas")
print(len(inputs_en))
print(len(outputs_pt))

tamanhos das entradas e saidas
208990
208990


## padding 

In [9]:
inputs_en = keras_preprocessing.sequence.pad_sequences(inputs_en, value=0, padding='post', maxlen=max_length)
outputs_pt = keras_preprocessing.sequence.pad_sequences(outputs_pt, value=0, padding='post', maxlen=max_length)

print(inputs_en[random.randint(0, len(inputs_en) - 1)])
print(outputs_pt[random.randint(0, len(outputs_pt) - 1)])

[8191   11  157    4  348   66  968   18  270   16 4663 7981 8192    0
    0]
[8116 7179   19  182 1566 7969 7905   34 1907 7906 8117    0    0    0
    0]


In [10]:
batch_size = 64
buffer_size = 20000

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((inputs_en, outputs_pt))
dataset = dataset.cache()
dataset = dataset.shuffle(buffer_size).batch(batch_size)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
print(dataset)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 15), dtype=tf.int32, name=None), TensorSpec(shape=(None, 15), dtype=tf.int32, name=None))>
